## Assessment - Data Manipulation and Data Cleaning

Given a csv file - Book.csv, which contains the information about books from the British Library.

Note: The csv file should be named as 'RR-DPL-AU2 - Book.csv' and stored in the same folder as this file.

#### 1. Read the csv in a data_frame.

In [7]:
# Import packages
import pandas as pd
import numpy as np

# Read csv file
books = pd.read_csv('RR-DPL-AU2 - Book.csv')

# Inspect dataframe column names
print(books.columns)

Index(['Identifier', 'Edition Statement', 'Place of Publication',
       'Date of Publication', 'Publisher', 'Title', 'Author', 'Contributors',
       'Corporate Author', 'Corporate Contributors', 'Former owner',
       'Engraver', 'Issuance type', 'Flickr URL', 'Shelfmarks'],
      dtype='object')


#### 2. Remove the following columns from the data_frame 
a.'Edition Statement',   
b.'Corporate Author',   
c.'Corporate Contributors',   
d.'Former owner',   
e.'Engraver',   
f.'Contributors',   
g.'Issuance type',   
h.'Shelfmarks'  

In [8]:
# Drop all required columns
columns_to_drop = ['Edition Statement', 'Corporate Author', 'Corporate Contributors', 'Former owner', 'Engraver', 'Contributors', 'Issuance type', 'Shelfmarks']
books = books.drop(columns_to_drop, axis = 1)

# Inspect column names again
print(books.columns)

Index(['Identifier', 'Place of Publication', 'Date of Publication',
       'Publisher', 'Title', 'Author', 'Flickr URL'],
      dtype='object')


#### 3.  Check the content of the column- 'Date of Publication’ and define a function to clean the value. E.g., the original value: 1879 [1878] to: 1879; [1858.] to 1858;

In [9]:
# count each value's occurance time and list the last 10 values
print(books['Date of Publication'].value_counts().tail(10))

# From the result we can see several problems:
# 1. description of reprint info after first publish year
# 2. not uniformed year format
# 3. noticed that all years are stored at the begining of each records

# Clean function to clean the column
# Only keep the first 4 digits from the beginning to get a year
def extract_year(column):
    # Define a regular expression (4 digits at the beginning, ignoring any square brackets)
    regex_year = r'^(\d{4})'
    column = column.str.extract(regex_year, expand = False)
    return column

# Run the function
books['Date of Publication'] = extract_year(books['Date of Publication'])

# Check again after cleaning
print(books['Date of Publication'].value_counts().tail(10))


1570 [reprinted 1844.]    1
1721                      1
[1814.]                   1
1875?]                    1
1892, [1891]              1
[1895-97.]                1
1838-39                   1
1854-55                   1
[1882, 85]                1
1834-43                   1
Name: Date of Publication, dtype: int64
1631    1
1725    1
1698    1
1663    1
1689    1
1700    1
1658    1
1632    1
1602    1
1708    1
Name: Date of Publication, dtype: int64


#### 4. Check the content of the column- ' Author’ and define a function to clean the value. And split the name to first name and last name. 

In [10]:
# Check content of column 'Author' 
print(books['Author'].head(20))

# Functions of split names
def split_name(col):
    first_names = []
    last_names = []
    for name in col:
        # Append NaN value if there's no name in the column
        if pd.isnull(name):
            first_names.append(np.nan)
            last_names.append(np.nan)
        else:
            names = name.split(",")
            # If the length of names is 1, it means there's no firstname
            if len(names) == 1:
                first_names.append(np.nan)
                last_names.append(names[0])
            else:
                last_names.append(names[0])
                first_names.append(names[1])
    full_names = pd.DataFrame({'First Name': first_names, 'Last Name': last_names})
    return full_names

# Apply the function, concat two new columns to the original dataframe
books = pd.concat([books, split_name(books['Author'])], axis = 1)

# Check new columns
print(books['First Name'])
print(books['Last Name'])

0                                                 A. A.
1                                             A., A. A.
2                                             A., A. A.
3                                             A., E. S.
4                                             A., E. S.
5                                             A., E. S.
6                                             A., F. E.
7                                         A., J.|A., J.
8                                                Remaʿ.
9                                                A., T.
10                                                  NaN
11                                         AALL, Jacob.
12    AAR, Ermanno - pseud. [i.e. Luigi Giuseppe Oro...
13                                                  NaN
14                            ABATE, Giovanni Agostino.
15                                    ABATI, Francesco.
16                        ABBADIE, Antoine Thompson d'.
17                        ABBADIE, Antoine Thomp

#### 5. Check the content of the column- ‘Title’ and define a function to clean the value. E.g., the original value: Walter Forbes. [A novel.] By A. A to: Walter Forbes;  Love the Avenger. By the author of “All for Gr.. to Love The Avenger.

In [11]:
# Check first 20 values of 'Title' column
print(books['Title'].head(20))

# Noted that we don't need things in square brackets and author name after 'By'
# Titles are usually in the first part of the value, ended with period mark
# Some titles is stored in the square brackets

# Function to clean the title
def extract_book_title(col):
    col = col.apply(lambda x: x.split('.')[0].strip('[]'))
    return col
   
# Execute the function and store it to a new column 'Book Title'
books['Book Title'] = extract_book_title(books['Title'])

# Check the cleaned column
print(books['Book Title'].head(20))

0                     Walter Forbes. [A novel.] By A. A
1     All for Greed. [A novel. The dedication signed...
2     Love the Avenger. By the author of “All for Gr...
3     Welsh Sketches, chiefly ecclesiastical, to the...
4     [The World in which I live, and my place in it...
5     [The World in which I live, and my place in it...
6     Lagonells. By the author of Darmayne (F. E. A....
7     The Coming of Spring, and other poems. By J. A...
8     A Warning to the inhabitants of England, and L...
9     A Satyr against Vertue. (A poem: supposed to b...
10    An Account of the many and great Loans, Benefa...
11    Erindringer som Bidrag til Norges Historie fra...
12    Gli Studi storici in terra d'Otranto ... Framm...
13    De Aardbol. Magazijn van hedendaagsche land- e...
14    Cronache Savonesi dal 1500 al 1570 ... Accresc...
15    See-Saw; a novel ... Edited [or rather, writte...
16    Géodésie d'une partie de la Haute Éthiopie,...
17                                  [With eleven

#### 6. Check the content of the column- ‘Place of Publication’ and define a function to clean the value. E.g., the original value: London; Virtue & Yorston to: London.

In [12]:
# Inspect first 60 values of 'Place of Publication'
print(books['Place of Publication'].head(60))

# Noted several problems:
# 1. Places contain 'London' have unwated elements
# 2. Patterns like pp. 40. G. Bryan & Co: Oxford, 1898, where city is between ':' and ','
# 3. Places doesn't have a valid city, only numbers (1845), should be replaced with Nan

# Functions for extract cities from the column
def extract_cities(col):
    cities = []
    for place in col:
        # location for find cities in patterns like "pp. 40. G. Bryan & Co: Oxford, 1898"
        colon_idx = place.find(":")
        comma_idx = place.find(",")
        # If find a record type like the pattern above, extract the city
        if colon_idx != -1:
            city = place[colon_idx+1:comma_idx]
        # If not, find any records contains 'London' and replace the record with only 'London'
        elif 'London' in place:
            city = 'London'
        else:
            city = place
        cities.append(city)
    col = pd.Series(cities)
    # Check for non-valid locations and replace it with Nan value
    col = np.where(col.str.isnumeric(), np.nan, col)
    return col

# Run the function and clean the column
books['Place of Publication'] = extract_cities(books['Place of Publication'])

# Check cleaned column
print(books['Place of Publication'].head(60))


0                                  London
1                London; Virtue & Yorston
2                                  London
3                                  London
4                                  London
5                                  London
6                                  London
7     pp. 40. G. Bryan & Co: Oxford, 1898
8                                 London]
9                                  London
10                               Coventry
11                            Christiania
12                                Firenze
13                              Amsterdam
14                                 Savona
15                                 London
16                                  Paris
17                                  Paris
18                            Puerto-Rico
19                               New York
20                                   Hull
21                                 London
22                                 Oxonii
23                                